In [1]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2024-12-05 11:30:29.802557: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-12-05 11:30:29.802599: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-12-05 11:30:29.842761: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 11:30:30.798208: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-05 11:30:30.798309: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: ca

2.10.0


In [15]:
(train_images, train_labels), (
    test_images,
    test_labels,
) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [16]:
# Define a simple sequential model
def create_model():
    model = tf.keras.Sequential(
        [
            keras.layers.Dense(512, activation="relu", input_shape=(784,)),
            keras.layers.Dropout(0.2),
            keras.layers.Dense(10),
        ]
    )

    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
    )

    return model


# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [17]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, save_weights_only=True, verbose=1
)

# Train the model with the new callback
model.fit(
    train_images,
    train_labels,
    epochs=10,
    validation_data=(test_images, test_labels),
    callbacks=[cp_callback],
)  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
25/32 [======================>.......] - ETA: 0s - loss: 1.2815 - sparse_categorical_accuracy: 0.6438
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 14ms/step - loss: 1.1719 - sparse_categorical_accuracy: 0.6720 - val_loss: 0.6925 - val_sparse_categorical_accuracy: 0.7890
Epoch 2/10
32/32 [==============================] - ETA: 0s - loss: 0.4107 - sparse_categorical_accuracy: 0.8840
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 0.4107 - sparse_categorical_accuracy: 0.8840 - val_loss: 0.5482 - val_sparse_categorical_accuracy: 0.8340
Epoch 3/10
18/32 [===============>..............] - ETA: 0s - loss: 0.2943 - sparse_categorical_accuracy: 0.9236
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 7ms/step - loss: 0.2878 - sparse_categorical_accuracy: 0.9260 - val_loss: 0.4921 - val_sparse_categorical_accuracy: 0.8410
Epoch 4/10
29/32 [=========

In [18]:
os.listdir()

['SpikeLussac.ipynb',
 'ResultsLoadingDat.ipynb',
 'ModelLoading.ipynb',
 'debug.ipynb',
 'training_1',
 'SpikeLoading.ipynb',
 'training_2',
 '.ipynb_checkpoints',
 'ResultsLoading.ipynb']

In [19]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

In [20]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.4169 - sparse_categorical_accuracy: 0.0590 - 177ms/epoch - 6ms/step
Untrained model, accuracy:  5.90%


In [26]:
checkpoint_path

'training_2/cp-{epoch:04d}.ckpt'

In [21]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4054 - sparse_categorical_accuracy: 0.8670 - 89ms/epoch - 3ms/step
Restored model, accuracy: 86.70%


In [22]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Calculate the number of batches per epoch
import math

n_batches = len(train_images) / batch_size
n_batches = math.ceil(
    n_batches
)  # round up the number of batches to the nearest whole integer

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, verbose=1, save_weights_only=True, save_freq=5 * n_batches
)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(
    train_images,
    train_labels,
    epochs=50,
    batch_size=batch_size,
    callbacks=[cp_callback],
    validation_data=(test_images, test_labels),
    verbose=0,
)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [23]:
os.listdir(checkpoint_dir)

['cp-0015.ckpt.index',
 'checkpoint',
 'cp-0020.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0025.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0010.ckpt.index',
 'cp-0005.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index']

In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [24]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4852 - sparse_categorical_accuracy: 0.8760 - 188ms/epoch - 6ms/step
Restored model, accuracy: 87.60%
